<a href="https://colab.research.google.com/github/nosadaniel/BodyMassIndex-flutter-app/blob/master/anti_phishing_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install core LangChain/LangGraph/LangSmith, LLM support, and utilities
!pip install -qU langgraph langchain langsmith langchain_core langchain_google_genai langchain_community langchain_ollama langchain_mistralai
# Install data analysis and visualization libraries
!pip install -q matplotlib seaborn plotly scikit-learn pandas numpy

# Install some additional libraries for data processing and visualization
!pip install requests json-fix beautifulsoup4

# instal datasets
!pip install -qU datasets

#install the URl checking tool
!pip install -qU sec-mcp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.9/367.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.6/441.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 14.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently ta

In [ ]:
# update the sec-mcp blacklist database
!sec-mcp update

#check sec-mcp status(optional)
!sec-mcp status

2025-07-17 04:59:12,605 - httpx - INFO - HTTP Request: GET https://www.spamhaus.org/drop/drop.txt "HTTP/1.1 200 OK"
2025-07-17 04:59:12,613 - sec_mcp.update_blacklist - INFO - First 5 parsed entries for SpamhausDROP: [(None, '1.10.16.0/20', '2025-07-17 04:59:12', 8, 'SpamhausDROP'), (None, '1.19.0.0/16', '2025-07-17 04:59:12', 8, 'SpamhausDROP'), (None, '1.32.128.0/18', '2025-07-17 04:59:12', 8, 'SpamhausDROP'), (None, '2.56.192.0/22', '2025-07-17 04:59:12', 8, 'SpamhausDROP'), (None, '2.57.122.0/24', '2025-07-17 04:59:12', 8, 'SpamhausDROP')]
2025-07-17 04:59:21,739 - sec_mcp.update_blacklist - INFO - Updated SpamhausDROP: 0 URLs, 0 domains, 1544 IPs.
2025-07-17 04:59:21,741 - httpx - INFO - HTTP Request: GET https://raw.githubusercontent.com/openphish/public_feed/refs/heads/main/feed.txt "HTTP/1.1 200 OK"
2025-07-17 04:59:21,744 - httpx - INFO - HTTP Request: GET https://data.phishtank.com/data/online-valid.csv "HTTP/1.1 302 Found"
2025-07-17 04:59:21,753 - sec_mcp.update_blacklist -

Get api keys

In [ ]:
from google.colab import userdata
google_api_key = userdata.get('GOOGLE_API_KEY')
langsmith_endpoint = userdata.get('LANGSMITH_ENDPOINT')
langsmith_api_key = userdata.get('LANGSMITH_API_KEY')
langsmith_tracing = userdata.get('LANGSMITH_TRACING')
langsmith_project = userdata.get('LANGSMITH_PROJECT')
mistral_api_key = userdata.get('MISTRAL_API_KEY')


In [110]:
# getting secrets from userdata and setting it into the global environment
import os

# Assuming the variables are retrieved in a previous cell (e.g., 8crHJ2gGsuc7)
# from google.colab import userdata
# langsmith_api_key = userdata.get('LANGSMITH_API_KEY')
# langsmith_tracing = userdata.get('LANGSMITH_TRACING') # Should be a string like "true" or "false"
# langsmith_project = userdata.get('LANGSMITH_PROJECT')
# langsmith_endpoint = userdata.get('LANGSMITH_ENDPOINT') # Optional, if not using default

if 'google_api_key' in globals() and google_api_key.low() == 'true':
    os.environ["GOOGLE_API_KEY"] = google_api_key
    print("Google API key set.")

if 'mistral_api_key' in globals() and mistral_api_key:
    os.environ["MISTRAL_API_KEY"] = mistral_api_key
    print("Mistral API key set.")
# Set LangSmith environment variables
if 'langsmith_tracing' in globals() and langsmith_tracing.lower() == 'true':
    os.environ["LANGCHAIN_TRACING"] = "true"
    print("LangSmith tracing enabled.")
else:
    os.environ["LANGCHAIN_TRACING"] = "false"
    print("LangSmith tracing disabled.")

if 'langsmith_api_key' in globals() and langsmith_api_key:
    os.environ["LANGCHAIN_API_KEY"] = langsmith_api_key
    print("LangSmith API key set.")
else:
    print("LangSmith API key not found or empty. Tracing may not work.")


if 'langsmith_project' in globals() and langsmith_project:
    os.environ["LANGCHAIN_PROJECT"] = langsmith_project
    print(f"LangSmith project set to: {langsmith_project}")
else:
     print("LangSmith project name not found or empty. Using default project.")


if 'langsmith_endpoint' in globals() and langsmith_endpoint:
    os.environ["LANGCHAIN_ENDPOINT"] = langsmith_endpoint
    print(f"LangSmith endpoint set to: {langsmith_endpoint}")
# else: Using default LangSmith endpoint

LangSmith tracing enabled.
LangSmith API key set.
LangSmith project set to: Anti_phishing_fine_tuning
LangSmith endpoint set to: https://api.smith.langchain.com


In [111]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import ChatOllama
from langchain_mistralai import ChatMistralAI
def get_llm(llm_provider_name):
  try:
    llm = None
    if llm_provider_name == 'gemini':
      llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash", temperature=0.0, google_api_key=google_api_key)
    if llm_provider_name == "ollama":
      llm = ChatOllama(
        model="llama3.2:3b", temperature=0.0, base_url="http://127.0.0.1:11434")
    if llm_provider_name == "mistral":
      llm = ChatMistralAI(
        model="mistral-large-latest", temperature=0, max_retries=2)
    if llm is None:
      raise Exception("Invalid LLM provider name")
    return llm
  except Exception as e:
    print(f"Error getting LLM: {e}")
    raise e

# example
llm = get_llm('gemini')
if llm is not None:
  print(f"LLM: {llm}")
  messages = [
                ("system", "Translate the user sentence to French."),
                ("human", "I love programming."),
            ]
  result = llm.invoke(messages)

  print(result)


LLM: model='models/gemini-1.5-flash' google_api_key=SecretStr('**********') temperature=0.0 client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7f58638a2150> default_metadata=() model_kwargs={}
content="J'adore programmer." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run--fa664789-bdef-473d-9419-8964dab08536-0' usage_metadata={'input_tokens': 11, 'output_tokens': 6, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}}


show available dataset

In [112]:
from pathlib import Path
from typing import List
def get_all_available_datasets()->List[str]:
  # Define path to the dataset directory
  dataset_dir = Path('drive/MyDrive/dataset/email_phishing')
  #list available datasets
  available_datasets = [file for file in dataset_dir.glob('*.csv')]
  return available_datasets

available_datasets = get_all_available_datasets()
for dataset in available_datasets:
    print(f"Available dataset: {dataset}")

Available dataset: drive/MyDrive/dataset/email_phishing/Nazario.csv
Available dataset: drive/MyDrive/dataset/email_phishing/SpamAssasin.csv
Available dataset: drive/MyDrive/dataset/email_phishing/CEAS_08.csv
Available dataset: drive/MyDrive/dataset/email_phishing/Nigerian_Fraud.csv


Load dataset

In [113]:
from datasets import load_dataset

def load_local_dataset(type:str, path:str):
  try:
    dataset = load_dataset(type, data_files=path,split='train')
  except Exception as e:
    print(f"Error loading dataset: {e}")
    return None
  return dataset


#load dataset from google drive
dataset_file_names = get_all_available_datasets()
nigeria_phishing_dataset = dataset_file_names[3]
dataset = load_local_dataset('csv',f'{nigeria_phishing_dataset}')
if dataset is not None:
  selected_dataset = dataset.select(range(10))
  print(selected_dataset)

Dataset({
    features: ['sender', 'receiver', 'date', 'subject', 'body', 'urls', 'label'],
    num_rows: 10
})


# Some Helpers Functions

In [114]:
import re

def extract_email_username(email_string) -> tuple[str | None, str | None ]:
  """
  Extracts email and username (name part before the email) from a string.

  Args:
    email_string: The input string.

  Returns:
    A tuple containing the email and username (name), or (None, None) if not found.
  """
  # Add a check for None input
  if email_string is None:
    return None, None

  # Regex to capture the name part before the email in angle brackets
  match = re.search(r'^(.*?)\s*<([\w\.\-]+@[\w\.\-]+)>', email_string)
  if match:
    name = match.group(1).strip() # Extract the name part and remove leading/trailing whitespace
    email = match.group(2) # Extract the email part
    return email, name
  # If the above pattern doesn't match, try to extract just an email without a name part
  email_match = re.search(r'<([\w\.\-]+@[\w\.\-]+)>', email_string)
  if email_match:
      email = email_match.group(1)
      return email, None # Return email and None for username if no name is found
  # If neither pattern matches, return None for both
  return None, None

#example
for data in selected_dataset:
  email,username = extract_email_username(data['sender'])
  if email and username:
    print(email,username)
  elif email:
    print(email, "No username found")
  else:
    print(data['sender'], "No email or username found")

james_ngola2002@maktoob.com MR. JAMES NGOLA.
bensul2004nng@spinfinder.com Mr. Ben Suleman
obong_715@epatra.com PRINCE OBONG ELEME
obong_715@epatra.com PRINCE OBONG ELEME
m_abacha03@www.com Maryam Abacha
davidkuta@postmark.net Kuta David
tunde_dosumu@lycos.com Barrister tunde dosumu
william2244drallo@maktoob.com William Drallo
abdul_817@rediffmail.com MR USMAN ABDUL
barrister_td@lycos.com Tunde Dosumu


In [115]:

def extract_url_from_body(message_body: str,is_url:int) -> str | None:
  """
  Extracts a URL from a string only when is_url is true.

  Args:
    message_body: The input string.

  Returns:
    The extracted URL, or None if not found.
  """
  if (is_url == 1):
      url_match = re.search(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', message_body)
      if url_match:
          return url_match.group(0)
  return None

#example
for data in selected_dataset:
  url = extract_url_from_body(data['body'],data['urls'])
  if url:
    print(url)


http://login.mail.lycos.com/brandPage.shtml?pageId=plus&ref=lmtplus
http://login.mail.lycos.com/brandPage.shtml?pageId=plus&ref=lmtplus


In [116]:
import re
from typing import List # Import List for type hinting

def extract_email_from_body(message_body: str) -> List[str]:
  """
  Extracts all unique email addresses from a message body using a comprehensive regex.

  Args:
    message_body: The input string (message body).

  Returns:
    A list of unique extracted email addresses. Returns an empty list if none are found.
  """
  # Add a check for None input
  if message_body is None:
      return []

  # More comprehensive regex for email extraction using findall
  email_list = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', message_body)

  # Remove duplicates by converting to a set and back to a list
  unique_emails = list(set(email_list))

  return unique_emails

# Example usage with the selected_dataset
for data in selected_dataset:
  emails = extract_email_from_body(data['body'])
  if emails:
    print(f"Unique Emails extracted: {emails}")

Unique Emails extracted: ['james_ngola2002@maktoob.com']
Unique Emails extracted: ['obong_715@epatra.com']
Unique Emails extracted: ['obong_715@epatra.com']
Unique Emails extracted: ['tony_m@lawyer.com']
Unique Emails extracted: ['davidkuta2002@yahoo.com']
Unique Emails extracted: ['barrister_tunde@lawyer.com']
Unique Emails extracted: ['william2244@maktoob.com', 'william22drallo@yahoo.com']
Unique Emails extracted: ['barrister_tunde@lawyer.com']


In [117]:
def is_phishing(label:int)->bool:
  if(label == 1):
    return True
  return False

#example
for data in selected_dataset:
  is_phishing_email = is_phishing(data['label'])
  print(f"is_phishing: {is_phishing_email}")

is_phishing: True
is_phishing: True
is_phishing: True
is_phishing: True
is_phishing: True
is_phishing: True
is_phishing: True
is_phishing: True
is_phishing: True
is_phishing: True


Define a structure data for the email database

In [118]:
#define class model
from typing import List, Optional
from pydantic import BaseModel
class EmailDataSet(BaseModel):
  sender_name: str
  sender_email: str | None
  receiver_email: str
  date: str
  subject: str
  body: str
  urls: List[str]
  extra_emails: List[str]
  is_phishing: bool


class UrlCheckResult(BaseModel):
  url: str
  is_malicious: bool
  confidence: float # 0.0 to 1.0
  details: Optional[str] # Explanation from sec-mcp

class EmailAndUrlData(BaseModel):
  email_data: EmailDataSet
  url_checker_results: List[UrlCheckResult]

format the dataset and return a list of structured email data


In [119]:
from typing import List
from datasets import Dataset

def format_dataset(dataset: Dataset) -> List[EmailDataSet]:
  emails_data: List[EmailDataSet] = [] # Initialize as an empty list
  try:
      for data in dataset:
        urls = []  # Initialize for each email
        # emails_in_body = [] # Initialize for each email - This will now be assigned the list directly

        # Use .get() with a default value for potentially missing or None fields
        # Use "" for strings, [] for lists, 0 for integers, False for booleans etc.
        sender_val = data.get('sender', '')
        sender_email, sender_name = extract_email_username(sender_val) # Pass the potentially default value to the helper

        receiver_email_val = data.get("receiver", "") # Get value with default
        date_received_val = data.get("date", "") # Get value with default
        subject_val = data.get("subject", "") # Get value with default
        body_val = data.get("body", "") # Get value with default
        urls_flag_val = data.get('urls', 0) # Default to 0 for urls flag
        label_val = data.get('label', 0) # Default to 0 for label


        url_inside_body = extract_url_from_body(body_val, urls_flag_val) # Use the potentially defaulted body_val and urls_flag_val

        is_phish = is_phishing(label_val) # Use the potentially defaulted label_val


        if url_inside_body:
          urls.append(url_inside_body)

        # Use the updated extract_email_from_body function
        emails_in_body = extract_email_from_body(body_val) # Use the potentially defaulted body_val
        # No need to append to emails_in_body here as extract_email_from_body already returns a list

        email_data_item = EmailDataSet( # Use a different variable name to avoid confusion
          sender_name=sender_name if sender_name is not None else "", # Ensure sender_name is string or ""
          sender_email=sender_email if sender_email is not None else "", # Ensure sender_email is string or ""
          receiver_email=receiver_email_val if receiver_email_val is not None else "", # Explicitly check and default
          date=date_received_val if date_received_val is not None else "", # Explicitly check and default
          subject=subject_val if subject_val is not None else "", # Explicitly check and default
          body=body_val if body_val is not None else "", # Explicitly check and default
          urls=urls, # urls list will be handled correctly even if body was None
          extra_emails=emails_in_body, # Assign the list of unique emails directly, handled by extract_email_from_body's None check
          is_phishing=is_phish
        )
        emails_data.append(email_data_item)

      return emails_data
  except Exception as e:
    error = f"Error formatting dataset: {e}"
    print(error)
    return []


# examples
formatted_dataset = format_dataset(selected_dataset)
for data in formatted_dataset:
  if(data.urls):
    print(f"only email with urls => {data.urls}")


only email with urls => ['http://login.mail.lycos.com/brandPage.shtml?pageId=plus&ref=lmtplus']
only email with urls => ['http://login.mail.lycos.com/brandPage.shtml?pageId=plus&ref=lmtplus']


Function to check if urls are malicious

In [120]:
from operator import is_
from sec_mcp import SecMCP
# initialize SecMCP once
sec_mcp_checker = SecMCP()
def check_malicious_urls(urls: List[str]) -> List[UrlCheckResult]:
  """check a list of URLs against security blacklist using the sec_mcp
    library. Return a list of check results, one for each URL.
  """
  if not urls: # Corrected condition to check if the list is empty
    return []

  #print(f"checking {len(urls)} URLs: {urls}")
  results = []
  try:
    # use batch checking for efficiency
    batch_results = sec_mcp_checker.check_batch(urls)
    for i, url in enumerate(urls):
      #
      if i < len(batch_results):
        check_result = batch_results[i]
        is_malicious = check_result.blacklisted
        confidence = 1.0 if is_malicious else 0.5
        details = check_result.explanation
        results.append({
          'url': url,
          'is_malicious': is_malicious,
          'confidence': confidence,
          'details': details
        })
      else: # Handle potential mismatch in results length (unlikely but safe)
          results.append({
            'url': url,
            'is_malicious': False,
            'confidence': 0.0,
            'details': "Check result unavalable"
          })
    return results
  except Exception as e:
    # Return default safe results on error
    error_results = [{
      'url': url,
      'is_malicious': False,
      'confidence': 0.0,
      'details': f"Error checking URL: {str(e)}"
    } for url in urls]
    print(f"Error checking URL: {error_results}")
    return error_results

# Example Test:
formatted_dataset = format_dataset(selected_dataset)
# Flatten the list of lists of URLs into a single list of URLs
urls_to_check = [url for data in formatted_dataset for url in data.urls]
check_urls_result = check_malicious_urls(urls_to_check)
for result in check_urls_result:

  print(f"url_checker => {result}")
# or do this
#for data in formatted_dataset:
 # if data.urls:
  #  urls_results = check_malicious_urls(data.urls)
   # print(f"urls results: {urls_results}")

url_checker => {'url': 'http://login.mail.lycos.com/brandPage.shtml?pageId=plus&ref=lmtplus', 'is_malicious': False, 'confidence': 0.5, 'details': 'Not blacklisted'}
url_checker => {'url': 'http://login.mail.lycos.com/brandPage.shtml?pageId=plus&ref=lmtplus', 'is_malicious': False, 'confidence': 0.5, 'details': 'Not blacklisted'}


Get Results

In [121]:
def full_email_data(dataset) ->List[EmailAndUrlData]:
  try:
    results =  []
    formatted_dataset = format_dataset(dataset)
    # Flatten the list of lists of URLs into a single list of URLs
    for data in formatted_dataset:
      if data.urls:
        #check for malicious url
        check_urls_result = check_malicious_urls(data.urls)
        result_url = EmailAndUrlData(
            email_data=data,
            url_checker_results=check_urls_result
          )
        results.append(result_url)
      else:
        result = EmailAndUrlData(
            email_data=data,
            url_checker_results=[]
          )
        results.append(result)
    return results
  except Exception as e:
    print(f"Error getting results: {e}")
    return []

#example
results = full_email_data(selected_dataset)
for result in results:
    print(f"email_data: {result.email_data}")
    print(f"url_checker_result: {result.url_checker_results}")





email_data: sender_name='MR. JAMES NGOLA.' sender_email='james_ngola2002@maktoob.com' receiver_email='webmaster@aclweb.org' date='Thu, 31 Oct 2002 02:38:20 +0000' subject='URGENT BUSINESS ASSISTANCE AND PARTNERSHIP' body='FROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-27-587908.\nE-MAIL: (james_ngola2002@maktoob.com).\n\nURGENT BUSINESS ASSISTANCE AND PARTNERSHIP.\n\n\nDEAR FRIEND,\n\nI AM ( DR.) JAMES NGOLA, THE PERSONAL ASSISTANCE TO THE LATE CONGOLESE (PRESIDENT LAURENT KABILA) WHO WAS ASSASSINATED BY HIS BODY GUARD ON 16TH JAN. 2001.\n\n\nTHE INCIDENT OCCURRED IN OUR PRESENCE WHILE WE WERE HOLDING MEETING WITH HIS EXCELLENCY OVER THE FINANCIAL RETURNS FROM THE DIAMOND SALES IN THE AREAS CONTROLLED BY (D.R.C.) DEMOCRATIC REPUBLIC OF CONGO FORCES AND THEIR FOREIGN ALLIES ANGOLA AND ZIMBABWE, HAVING RECEIVED THE PREVIOUS DAY (USD$100M) ONE HUNDRED MILLION UNITED STATES DOLLARS, CASH IN THREE DIPLOMATIC BOXES ROUTED THROUGH ZIMBABWE.\n\nMY PURPOSE OF WRITING YOU THIS LETTER IS TO SOLICIT

Format the structured email data list of string to be used in training


In [122]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage
def get_prompt(data: EmailAndUrlData) -> List[str]:
  # example of system message
  system_message = """
    We are building a anti-phishing email service.
    you are being exposed to data set of emails
    Your goal is to generate confidence score (trusted_score: 0-100, where 0 is
    highly likely phishing,is highly likely legitimate),
    explanation: reasoning for the confidence score,
    recommendation: actionable advice,
    url_list: urls originally present in the dataset
    url_check_result: if available in the data set
    is_phishing: if the email is phishing or not base on the dataset


    Analysis Steps for generaion confidence score, explanation and recommendations:
      1. Examine the sender address: Doest it look authentic? Spoofed? or from
        a suspicious domain?
      2. Examine any additional email address if found on the email: are they legitimate.
      3. Analyze the subject line: Urgency? Generic greetings? weird characters?
        or Misspellings?
      4. Read the body content: Look for urgency/pressure tactics, grammar/spelling
        errors, requests fro sensitive information (credentials), suspicious offers,
        mismatched links (text vs actual URL if available).
      5. Synthesize findings: Combine evidence from sender, subject, body, and URL
        checks. A single indicator might not be conclusive, but multiple red flags
        increase the likelihood of phishing.
  """
  # example of response from LLM that the email is phishing
  ai_message = """
        {
          'is_phishing': true,
          'trusted_score': 80,
          'explanation': 'Red flags: spoofed sender address, generic subject line, suspicious URL with low confidence.',
          'recommendation': "Do not click on the link or respond to the email. Report it as spam and block the sender's IP address if possible.",
          'url_list': ['http://www.microsoftware.com/login?username&password'],
          'url_check_results': [{'url': 'http://www.microsoftware.com/login?username&password', 'is_malicious': False, 'confidence': 0.5, 'details': ' blacklisted'}],
        }
  """
  # example of a phishing email
  human_sample_data = """
    EMAIL_CONTENT:
      sender_name: John Joe
      sender_email: john.Joe@microsoft.com
      subject: Important Account Update Subject: Your account has been compromised
      body: We have recently detected suspicious activity on your account. To ensure the security of your account, we need you to update your password immediately.
      Please click on the link below to update your password: http://www.microsoftware.com/login?username&password
      After updating your password, please log in to your account and check for any unusual activity. If you notice any suspicious transactions or unauthorized access, please contact us immediately.
      Thank you for your cooperation in maintaining the security of your account.
      Please send your response to this email john.joe@gmail.com
      Best regards,
      John Joe
      Senior IT Support
      Microsoft Inc.
      date: 2023-08-01

    OPTIONAL URLs ANALYSIS RESULT:
    [{'url': ''http://www.microsoftware.com/login?username&password', 'is_malicious': True, 'confidence': 1.0, 'details': 'blacklisted'}]

    ADDITIONAL EMAIL ADDRESS
    [{john.joe@gmail.com}]

    EMAIL_IS_PHISHING: True


  """

  # real email data from a dataset
  human_dataset = f"""
  EMAIL_CONTENT:
    sender_name: {data.email_data.sender_name}
    sender_email: {data.email_data.sender_email}
    subject: {data.email_data.subject}
    body: {data.email_data.body} send reply to this {data.email_data.extra_emails}
    click this link  {data.email_data.urls}
    date: {data.email_data.date}

  OPTIONAL URLs ANALYSIS RESULT:
    {data.url_checker_results}

  ADDITIONAL EMAIL ADDRESS
    {data.email_data.extra_emails}

  EMAIL_IS_PHISHING:
    {data.email_data.is_phishing}

  RESPONSE STRUCTURE:

  """

  messages = [
      SystemMessage(content=system_message),
      HumanMessage(content=human_sample_data),
      AIMessage(content=ai_message),
      HumanMessage(content=human_dataset)
  ]
  return messages

#example
results = full_email_data(selected_dataset)

for result in results:
  input = get_prompt(result)
  print(input)


[SystemMessage(content='\n    We are building a anti-phishing email service. \n    you are being exposed to data set of emails\n    Your goal is to generate confidence score (trusted_score: 0-100, where 0 is\n    highly likely phishing,is highly likely legitimate), \n    explanation: reasoning for the confidence score,\n    recommendation: actionable advice,\n    url_list: urls originally present in the dataset\n    url_check_result: if available in the data set\n    is_phishing: if the email is phishing or not base on the dataset\n    \n\n    Analysis Steps for generaion confidence score, explanation and recommendations:\n      1. Examine the sender address: Doest it look authentic? Spoofed? or from\n        a suspicious domain?\n      2. Examine any additional email address if found on the email: are they legitimate.\n      3. Analyze the subject line: Urgency? Generic greetings? weird characters? \n        or Misspellings?\n      4. Read the body content: Look for urgency/pressure t

use llm to generate the response

In [123]:
from langgraph.prebuilt import create_react_agent
def create_agent(llm):
  try:
    agent = create_react_agent(model=llm, tools=[])
    return agent
  except Exception as e:
    print(f"Error creating agent: {e}")
    return None


In [125]:
#run agent
llm = get_llm('gemini')
agent = create_agent(llm)
ai_response_data = []
if agent:
  results = full_email_data(selected_dataset)
  for result in results:
    messages = get_prompt(result)
    response = agent.invoke({"messages":messages})
    input_data = response['messages'][1].content
    ai_output = response['messages'][2].content

    ai_response_data.append({
      'user_input': input_data.content,
      'ai_output': ai_output.content
    })

print(f"training_data => {ai_response_data}")



  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 10
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 8
}
]

In [1]:
#save training data
import json
from typing import List, Dict

def save_training_data(data: List[Dict], filename: str = "training_data.jsonl") -> None:
  """
  Save training data to a JSONL file.

  Args:
    data: A list of dictionaries representing the training data.
    filename: The name of the JSONL file to save the data to. Default is "training_data.jsonl".
  """
  try:
    # Open the file in append mode ('a') to  add data without overwriting
    with open(filename, 'a') as file:
      # iterate through the data and write each item as a separate JSON line
      for item in data:
        json.dump(item, file)
        file.write('\n') # Add a newline to separate JSON objects
    print(f"Training data saved to {filename}")
  except Exception as e:
    print(f"Error saving training data: {e}")

# usage
with open(f'training_data.jsonl', 'w') as file:
   for data in ai_response_data:
      save_training_data([data], file)

